# Surface Event Pick Time

This is a modified version of the surface-event location+directivity analysis that Francesca Skene ( fskene@uw.edu), originally created by her in 7/22/22, who started as an undergraduate student at UW. This is marine denolle's version. It includes:
* Waveform download for each event on each volcano given the PNSN pick times of "su" events.
* Data pre-processing to trim the data within 2-12 Hz and remove outliers.
* phase picking using transfer-learned model (Ni et al, 2023)
* event location using 1D grid search
* directivity measurements (velocity and direction) using Doppler effects.
* gathering of the data into a CSV data frame.

Updated 01/24/2024
Marine Denolle
(mdenolle@uw.edu)

Import Modules

In [1]:
# import sys
# sys.path.append('/data/wsd01/pnwstore/')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import obspy
from obspy.core import UTCDateTime
from obspy.clients.fdsn.client import Client

import scipy
from scipy import optimize
from scipy.optimize import curve_fit
from geopy import distance
from datetime import datetime
from utils import *
from mbf_elep_func import *
import torch
plt.rcParams.update({'font.size': 10})


import seisbench.models as sbm
device = torch.device("cpu")

# from ELEP.elep.ensemble_statistics import ensemble_statistics
from ELEP.elep.ensemble_coherence import ensemble_semblance 
# from ELEP.elep.ensemble_learners import ensemble_regressor_cnn
from ELEP.elep import mbf, mbf_utils
from ELEP.elep import trigger_func

from ELEP.elep.mbf_utils import make_LogFq, make_LinFq, rec_filter_coeff, create_obspy_trace
from ELEP.elep.mbf import MB_filter as MBF

Parameters

In [2]:
# define clients to download the station data
# client = WaveformClient() # we ignore PNWdatastore for now
client2 = Client('IRIS')

t_before = 15 #number of seconds before pick time
# t_after = 15 #number of seconds after pick time
t_before_raw = 1200 #number of seconds before pick time before removing instrumental response
# t_after_raw = 1200 #number of seconds after pick time before removing instrumental response
fs = 40 #sampling rate that all waveforms are resampled to
window = 150 #window length of the signal (this will help with phase picking with EqT next). 
# Use 150 seconds @ 40 Hz gives 6001 points. 
pr = 98 #percentile
thr = 5 #SNR threshold
station_distance_threshold = 25
pi = np.pi
v_s = 1000 #shear wave velocity at the surface

# range of dates that we are looking at
t_beginning = UTCDateTime(2001,1,1,0,0,0) 
t_end = UTCDateTime(2023,8,2,23,59)

smooth_length = 5 # constant for smoothing the waveform envelopes
low_cut = 1 #low frequency threshold
high_cut = 12 #high frequency threshold
# az_thr = 1000 #threshold of distance in meters from source location
# step = 100 #step every 100 m
# t_step = 1 #step every second
# ratio = 5.6915196 #used to define the grid 
# colors = list(plt.cm.tab10(np.arange(10)))*3
# radius = 6371e3 # radius of the earth

## Volcano - Station Information

In [3]:
#this data includes all stations within 50km of each volcano and the lat, lon, elev of each station
df = pd.read_csv('../data/station/Volcano_Metadata_50km.csv')

## PNSN SU Pick information

In [4]:
f1 = pd.read_csv("../data/events/su_picks.txt",sep="|")
f1.head()
print(f1.keys())

Index(['date', 'net', 'sta', 'location', 'seedchan', 'iphase', 'quality',
       'orid', 'etype', 'evid'],
      dtype='object')


In [5]:
# clean up the spaces in the file
format='%Y/%m/%d %H:%M:%S'
test=f1["date"].values.tolist()
start_time_temp = [  datetime.strptime(x.strip(),'%Y/%m/%d %H:%M:%S') for x in f1["date"].values.tolist()]
# # Ignore events prior to t_beginning
ik=np.where(np.array(start_time_temp)>datetime(2001,1,1))[0][0]

# select only net, sta, evid, startime for event past the start date.

start_time = start_time_temp[ik:]
net=[ x.strip() for x in f1["net"].values.tolist()][ik:]
sta=[ x.strip() for x in f1["sta"].values.tolist()][ik:]
evt_id=[ x for x in f1["orid"].values.tolist()][ik:]
all_stas=set(sta)

## ML Models

In [6]:
import os
os.makedirs("/Users/marinedenolle/.seisbench/models/v3/eqtransformer",exist_ok=True)

In [7]:
!wget https://github.com/congcy/ELEP/raw/main/docs/tutorials/data/pnw.pt.v1 -O ~/.seisbench/models/v3/eqtransformer/pnw.pt.v1
!wget https://github.com/congcy/ELEP/raw/main/docs/tutorials/data/pnw.json.v1 -O ~/.seisbench/models/v3/eqtransformer/pnw.json.v1

--2024-01-26 11:33:55--  https://github.com/congcy/ELEP/raw/main/docs/tutorials/data/pnw.pt.v1
Resolving github.com (github.com)... 20.29.134.23
Connecting to github.com (github.com)|20.29.134.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/congcy/ELEP/main/docs/tutorials/data/pnw.pt.v1 [following]
--2024-01-26 11:33:55--  https://raw.githubusercontent.com/congcy/ELEP/main/docs/tutorials/data/pnw.pt.v1
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1592020 (1.5M) [application/octet-stream]
Saving to: ‘/Users/marinedenolle/.seisbench/models/v3/eqtransformer/pnw.pt.v1’

/Users/marinedenoll 100%[===================>]   1.52M  9.53MB/s    in 0.2s    

2024-01-26 11:33:55 (9.53 MB/s) - ‘/Users/m

In [8]:
# download models
list_models_name = ["pnw","ethz","instance","scedc","stead","geofon"]
pn_pnw_model = sbm.EQTransformer.from_pretrained('pnw')
pn_ethz_model = sbm.EQTransformer.from_pretrained("ethz")
pn_instance_model = sbm.EQTransformer.from_pretrained("instance")
pn_scedc_model = sbm.EQTransformer.from_pretrained("scedc")
pn_stead_model = sbm.EQTransformer.from_pretrained("stead")
pn_geofon_model = sbm.EQTransformer.from_pretrained("geofon")
# pn_neic_model = sbm.EQTransformer.from_pretrained("neic")

list_models = [pn_pnw_model, pn_ethz_model, pn_instance_model, pn_scedc_model, pn_stead_model, pn_geofon_model]

pn_pnw_model.to(device);
pn_ethz_model.to(device);
pn_scedc_model.to(device);
# pn_neic_model.to(device);
pn_geofon_model.to(device);
pn_stead_model.to(device);
pn_instance_model.to(device);

In [9]:
paras_semblance = {'dt':0.025, 'semblance_order':4, 'window_flag':True, 
                   'semblance_win':0.5, 'weight_flag':'max'}
p_thrd, s_thrd = 0.01, 0.05

fqmin = low_cut
fqmax = high_cut
dt = 0.025; fs = 40
nfqs = 10
nt = 6000; nc = 3
fq_list = make_LogFq(fqmin, fqmax, dt, nfqs)
coeff_HP, coeff_LP = rec_filter_coeff(fq_list, dt)
MBF_paras = {'f_min':fqmin, 'f_max':fqmax, 'nfqs':nfqs, 'frequencies':fq_list, 'CN_HP':coeff_HP, 'CN_LP':coeff_LP, \
    'dt':dt, 'fs':fs, 'nt':nt, 'nc':nc, 'npoles': 2}

# Pick waveforms

In step, we repick all the waveforms, including those already picked by the network analysis.

In [16]:
from matplotlib.backends.backend_pdf import PdfPages
# pdf = PdfPages('../plots/MLPicks_test.pdf')
associated_volcano = "Mt_Rainier"
for n in range(len(evt_id)): 
    if start_time[n]<datetime(2023,11,1):continue   
    event_ID = str(evt_id[n])
    otime = UTCDateTime(start_time[n])
    networks=net[n]
    stations=sta[n]
    if sta[n]=="LON" or sta[n]=="LO2":continue
    # print(net[n],sta[n])
    try:
        associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
    except: 
        pass
    if associated_volcano!="Mt_Rainier":continue


    #get info for stations within 50km of volcano that event ocurred at
    stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
    networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
    latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
    longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
    elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()

    print("there are ",len(stations),"stations at",associated_volcano)           
    print(event_ID,otime)

#################### WAVEFORM DOWNLOAD #######################
    #Download all waveforms for that event based on stations and times
    bulk = [] 
    for m in range(0, len(networks)):
        bulk.append([networks[m], stations[m], '*', '*Z', otime-t_before_raw, otime+t_before_raw])
    try:
        st = client2.get_waveforms_bulk(bulk)
        if (len(st))<3:continue # if there are fewer than 3 stations, skip the event
        st = resample(st,fs)  #resampling the data to 40Hz for each trace

        evt_data = obspy.Stream()
        snr=[]
        stas=[]
        nets=[]
        centroid_time = []

        # #Keeping all traces for one event with channel z, SNR>10, and bandpassed between 2-12Hz
        # ,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
        for i,ii in enumerate(st):
            ii.detrend(type = 'demean')
            ii.filter('bandpass',freqmin=low_cut,freqmax=high_cut,corners=2,zerophase=True)
            # trim the data and noise window to exactly 6000 points
            signal_window = ii.copy()
            noise_window = ii.copy()
            signal_window.trim(otime - t_before, otime - t_before + window) # trim the signal at the first pick time of the PNSN data, with loose 40s before
            noise_window.trim(otime - window -t_before, otime - t_before) # noise window of the same length
            if not len(signal_window.data) or not len(signal_window.data): continue # skip if no data
            
            # if not np.percentile(np.abs(signal_window.data),pr):continue # skip if max amplitude is zero
            snr1 = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
                            / np.percentile(np.abs(noise_window.data),pr))/np.log(10))
            # print(snr1)
            # print(snr,max_amp_time)
            # signal_window.plot()x
            if snr1<thr: # and 100<max_amp_time<200:
                st.remove(ii)
                continue
            
            ############# Calculate envelope of the signal and pick centroid ############

################# ENVELOPE AND CENTROID #######################
            # enveloping the data 
            data_envelope = obspy.signal.filter.envelope(signal_window.data)
            data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
            data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
            print(data_envelope.shape)
            # finding the time of max amplitude of each event
            # signal_window is windowed at otime-t_v before the PNSN pick time
            print(np.abs(data_envelope))
            # crap = np.maxarg(np.abs(data_envelope))[0]+t_before # time of max amplitude relative to otime
            # centroid_time.append(crap)
            # max_amp = np.max(ii.data)                 
            # centroid_time.append(0)


            stas.append(ii.stats.station)
            nets.append(ii.stats.network)
            snr.append(snr1)
            evt_data.append(signal_window)
        print(stas)
        if len(stas)<3:continue
        break

    ################### ELEP #######################
        sta_available,ind = np.unique(np.array(stas),return_index=True)
        sta_available=sta_available[np.argsort(ind)]
        bigS = np.zeros(shape=(len(sta_available),3,6000))
        newstas=[]
        for i in range(len(sta_available)):
            stream = evt_data.select(station=sta_available[i])
            if len(stream[0].data)<6000:continue
            if len(stream)<3:
                # copy stream to 2 components, zero the missing data.
                tr3 = stream[0].copy() # assumed to be the vertical
                tr2 = stream[0].copy();tr2.stats.channel=stream[0].stats.channel[0:2]+"N"
                tr1 = stream[0].copy();tr1.stats.channel=stream[0].stats.channel[0:2]+"E"
                tr1.data=np.zeros(len(stream[0].data))
                tr2.data=np.zeros(len(stream[0].data))
                stream=obspy.Stream(traces=[tr1,tr2,tr3])
            # convert Stream into seisbench-friendly array    
            # fill in big array and order data ZNE
            bigS[i,0,:]=stream[2].data[:-1]
            bigS[i,1,:]=stream[1].data[:-1]
            bigS[i,2,:]=stream[0].data[:-1]
            newstas.append(sta_available[i])
        # pre process the data
        crap2  = bigS.copy()
        crap2 -= np.mean(crap2,axis=-1,keepdims=True) # demean data
        # original use std norm
        data_std = crap2 / np.std(crap2) + 1e-10
        # could use max data
        mmax= np.max(np.abs(crap2), axis=-1, keepdims=True)
        data_max = np.divide(crap2 ,mmax,out=np.zeros_like(crap2),where=mmax!=0)

            # test the new function
        smb_peak= apply_elep(evt_data, newstas, \
                list_models, MBF_paras, paras_semblance, t_before)
        if not max(smb_peak.shape):continue


################ SAVE PICKS ##############################
        # print(otime)
        # print("net,stas,snr,picks from ELEP,centroid pick time")
        for i in len(stas):
                print(nets[i],stas[i],snr[i],smb_peak[i]-t_before,centroid_time[i])

        # print(smb_peak-t_before) # this is the difference between the pick time and the PNSN pick time
        break

        # pick_mse[n] = np.sqrt(np.sum(smb_peak[:]-t_before)**2/len(stas))
        # pick_mae[n] = np.sum(np.abs(smb_peak[:]-t_before))/len(stas)

            ## plot figure

            # fig = plt.figure()#figsize = (11,8), dpi=200)
            # fig.suptitle(str(otime)+" "+associated_volcano)
            # ax = plt.subplot(1,1,1)
            # iplot = 0
            # for i in range(len(stas)):
            #     ax.plot(t-15,data_max[i,0,:]+iplot*1.5,linewidth=0.5)
            #     ax.plot(t-15,smb_pred[ i, :]/np.max(np.abs(smb_pred[ i, :]))+iplot*1.5,'k',linewidth=0.5)
            #     ax.set_yticks([])
            #     plt.text(-15, iplot*1.5+0.5, stas[i])
            #     # if i==ista:
            #     err_title=("%s %2.2f (s) error in picks"%(stas[i],smb_peak[i]-t_before))
            #     plt.text(60, iplot*1.5+0.5,err_title,color='r')
            #     plt.vlines(smb_peak[i]-t_before,iplot*1.5-1.,iplot*1.5+1.,'r')
            #     print(stas[i],smb_peak[i]-t_before)
            #     iplot+=1
            # # plt.grid(True)
            # ax.set_xlim([-t_before,80])
            # ax.set_xlabel('time (seconds) relative to PNSN picks')
            # plt.show()
            # pdf.savefig(fig)
            # plt.clf()
            # del fig

        # pick_mse[n] = np.sqrt(np.sum(smb_peak[:]-t_before)**2/len(stas))
        # pick_mae[n] = np.sum(np.abs(smb_peak[:]-t_before))/len(stas)
        # print(pick_mse[n],pick_mae[n] )
        # if n > 10000: break
    except:
        pass
# pdf.close()

there are  58 stations at Mt_Rainier
3247953 2023-11-05T07:05:01.000000Z
(1654,)
there are  58 stations at Mt_Rainier
3248053 2023-11-07T10:09:49.000000Z
(6001,)
there are  58 stations at Mt_Rainier
3248248 2023-11-08T23:16:43.000000Z
(6001,)
there are  58 stations at Mt_Rainier
3248253 2023-11-08T23:21:31.000000Z
(6001,)
there are  58 stations at Mt_Rainier
3248258 2023-11-08T23:24:03.000000Z


In [11]:
df[df["Volcano_Name"]=="Mt_Rainier"]

Unnamed: 0 Volcano_Name Network Station   Latitude   Longitude  Elevation  \
0            0   Mt_Rainier      CC    ARAT  46.788755 -121.852537   1823.137   
1            1   Mt_Rainier      CC    CARB  46.988320 -122.005410    872.470   
2            2   Mt_Rainier      CC    CRBN  46.988131 -121.961063    499.080   
3            3   Mt_Rainier      CC    GTWY  46.740163 -121.916765    617.235   
4            4   Mt_Rainier      CC    KAUT  46.730263 -121.857390    689.000   
5            5   Mt_Rainier      CC    MIRR  46.800468 -121.837353   1655.590   
6            6   Mt_Rainier      CC    OBSR  46.899719 -121.815331   2382.000   
7            7   Mt_Rainier      CC    OPCH  46.731319 -121.571171    592.430   
8            8   Mt_Rainier      CC    PANH  46.859032 -121.642592   2086.000   
9            9   Mt_Rainier      CC    PARA  46.786442 -121.742149   1651.180   
10          10   Mt_Rainier      CC    PR01  46.910390 -122.037770    648.000   
11          11   Mt_Rainier      CC    PR02  46.918305 -122.048639    461.000   
12          12   Mt_Rainier      CC    PR03  46.903438 -122.032271    532.000   
13          13   Mt_Rainier      CC    PR04  46.929694 -121.988976    912.700   
14          14   Mt_Rainier      CC    PR05  46.841649 -121.948912   1553.000   
15          15   Mt_Rainier      CC    RUSH  46.903060 -121.944390   1235.000   
16          16   Mt_Rainier      CC    SIFT  46.867092 -121.953246    720.900   
17          17   Mt_Rainier      CC    SR41  46.886680 -121.536642   1454.000   
18          18   Mt_Rainier      CC    STYX  46.963747 -122.080535    568.310   
19          19   Mt_Rainier      CC    TRON  46.997660 -122.174511    251.740   
20          20   Mt_Rainier      CC    VOIT  46.966330 -121.983400    977.100   
21          21   Mt_Rainier      UW     ASF  46.759880 -122.027771    528.000   
22          22   Mt_Rainier      UW     CMK  46.934479 -121.229782   1380.000   
23          23   Mt_Rainier      UW     CMM  46.490829 -122.013321    305.000   
24          24   Mt_Rainier      UW     COW  46.490829 -122.013321    305.000   
25          25   Mt_Rainier      UW     DLR  47.102650 -121.568619   1190.000   
26          26   Mt_Rainier      UW     FGM  47.075630 -121.763870   1158.000   
27          27   Mt_Rainier      UW     FMW  46.941399 -121.670998   1859.000   
28          28   Mt_Rainier      UW     GHW  47.041489 -122.273727    268.000   
29          29   Mt_Rainier      UW     GLK  46.557499 -121.610733   1305.000   
30          30   Mt_Rainier      UW     GRL  46.814030 -121.328072   1287.000   
31          31   Mt_Rainier      UW     GSM  47.203880 -121.796043   1313.000   
32          32   Mt_Rainier      UW     LMW  46.667831 -122.292549   1195.000   
33          33   Mt_Rainier      UW     LO2  46.750599 -121.809601    853.000   
34          34   Mt_Rainier      UW     LON  46.750599 -121.809601    853.000   
35          35   Mt_Rainier      UW     MHL  46.882210 -122.065231   1215.000   
36          36   Mt_Rainier      UW     MUP  46.839119 -121.126472   1456.000   
37          37   Mt_Rainier      UW     PAK  46.841320 -122.304268    436.000   
38          38   Mt_Rainier      UW    PCEP  47.111400 -122.291220    158.000   
39          39   Mt_Rainier      UW    PCMD  46.888962 -122.301483    239.000   
40          40   Mt_Rainier      UW    PUPY  46.495420 -122.146130    816.300   
41          41   Mt_Rainier      UW     RCM  46.835640 -121.732979   3085.000   
42          42   Mt_Rainier      UW     RCS  46.870831 -121.732307   2877.000   
43          43   Mt_Rainier      UW     RER  46.819050 -121.842133   1756.000   
44          44   Mt_Rainier      UW     RSH  46.812092 -121.529671   1770.000   
45          45   Mt_Rainier      UW     RSU  46.853161 -121.764259   4440.000   
46          46   Mt_Rainier      UW     RVC  46.944080 -121.974190   1000.000   
47          47   Mt_Rainier      UW     RVN  47.027222 -121.337837   1885.000   
48          48   Mt_Rainier      UW

In [12]:

# for n in range(11649,11660):    
#     event_ID = str(evt_id[n])
#     otime = UTCDateTime(start_time[n])
#     print(event_ID,otime)
#     if net != 'CN' and evt_id[n]!=evt_id[n-1]:
#         reference = str(net[n]+'.'+sta[n])
#         try:
#             associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
#         except: 
#             pass
#         print(associated_volcano)
            
#         #get info for stations within 50km of volcano that event ocurred at
#         stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
#         networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
#         latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
#         longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
#         elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()

#         if stations.count("LON")>0 and stations.count("LO2")>0:
#             index = stations.index("LO2")
#             del stations[index]
#             del networks[index]
#             del latitudes[index]
#             del longitudes[index]
#             del elevations[index]


# #################### WAVEFORM DOWNLOAD #######################
#         #Download all waveforms for that event based on stations and times
#         bulk = [] 
#         for m in range(0, len(networks)):
#             bulk.append([networks[m], stations[m], '*', '*H*', otime-t_before_raw, otime+t_after_raw])
#         st = client2.get_waveforms_bulk(bulk)

#         #remove unwanted data
#         for tr in st:
#             cha = tr.stats.channel
#             try:
#                 if len(tr.data)/tr.stats.sampling_rate < 239.9:
#                     st.remove(tr)
#             except:
#                 pass

#         #resampling the data to 40Hz for each trace
#         st = resample(st,fs) 
#         # #Plotting all traces for one event with channel z, SNR>10, and bandpasses between 2-12Hz
#         SNR,SNR_weight, no_weight,stas,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
#         fig = plt.figure()#figsize = (11,8), dpi=200)
#         fig.suptitle('evtID:UW'+ event_ID+associated_volcano)
#         evt_data = obspy.Stream()
#         # plt.rcParams.update({'font.size': 20})
#         ax = plt.subplot(1,1,1)
#         iplot = 0
#         # zz = evt_data.select(component="Z")
#         for i,ii in enumerate(st):
#             network = ii.stats.network
#             station = ii.stats.station
#             ii.detrend(type = 'demean')
#             ii.filter('bandpass',freqmin=low_cut,freqmax=high_cut,corners=2,zerophase=True)
#             cha = ii.stats.channel
#             # ii.trim(otime,otime+window)
#             starttime = ii.stats.starttime
#             signal_window = ii.copy()
#             noise_window = ii.copy()
#             signal_window.trim(otime - 10, otime - 10 + window) # trim the signal at the first pick time of the PNSN data, with loose 40s before
#             noise_window.trim(otime - window -10, otime - 10) # noise window of the same length
#             snr = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
#                             / np.percentile(np.abs(noise_window.data),pr))/np.log(10))
#             max_amp_time = np.argmax(noise_window.data)/fs
#             # print(snr,max_amp_time)
#             # signal_window.plot()x
#             if snr<thr: # and 100<max_amp_time<200:
#                 st.remove(ii)
#                 continue
#             t = signal_window.times()
#             t_diff[network+'.'+station] = starttime-otime 
#             # enveloping the data 
#             data_envelope = obspy.signal.filter.envelope(signal_window.data)
#             data_envelope /= np.max(data_envelope)
#             # data_envelope += iplot*1.5
#             # finding the time of max amplitude of each event
#             max_amp_times.append(max_amp_time)
#             max_amp = np.max(ii.data)      
#             # creating envelope data dictionary to calculate picktimes
#             data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
#             # kurt = rt.signal.kurtosis(signal_window.taper(max_percentage=0.05))
#             # kurt /= np.max(kurt)
#             data_env_dict[network+'.'+station]= data_envelope
#             # b,e = 115,150
#             if cha[-1]=="Z":
#                 ax.plot(t,signal_window.data/np.max(np.abs(signal_window.data))+iplot*1.5)
#                 ax.plot(t,data_envelope+iplot*1.5, color = 'k')
#                 # ax1.plot(t,kurt+iplot*1.5, color = 'r')
#                 # ax1.plot(t[(t_before_raw+b)*fs:(t_before_raw+e)*fs], data_envelope, color = 'k')
#                 # ax1.plot(t[b*fs:e*fs],ii.data[b*fs:e*fs]/np.max(np.abs(ii.data))+iplot*1.5)
#                 # ax1.plot(t[115*fs:150*fs], data_envelope, color = 'k')
#                 ax.set_xlabel('time (seconds)')
#                 # ax1.set_xlim([b+t_before_raw,e+t_before_raw])
#                 ax.set_yticks([])
#                 # plt.text(t[e*fs], iplot*1.5, 'SNR:'+str(int(snr)))
#                 # plt.text(t[b*fs], iplot*1.5, station)

#                 iplot = iplot+1
#             stas.append(ii.stats.station)
#             nets.append(ii.stats.network)
#             SNR.append(snr)
#             SNR_weight.append(int(snr))
#             no_weight.append(1)
#             evt_data.append(signal_window)
#     # impose at least 4 stations
#     if len(evt_data)<4:  
#         continue

#     sta_available = np.unique(np.array(stas))
#     print(sta_available)
    

# ## Get an approximative measurement of duration taking the highest SNR data
#     ik=np.argmax(SNR)
#     crap =np.cumsum(np.abs(evt_data[ik].data))
#     maxcrap = crap[-1]
#     Td=len(np.where((crap>=0.02*crap[-1])&(crap<=0.95*crap[-1]))[0])/fs
#     print("The approximate duration of this event is %f s "%Td)



# ############## PEAK FREQUENCY MEASUREMENTS ############
# # Given the approximate measurement of duration, window the signal windows around that
# # then measure peak frequency so that there is less noise in it.
# # perform this on the Z component only.
#     data2measure_peak_frequency=evt_data.copy()
#     print(data2measure_peak_frequency[3:6])
#     data2measure_peak_frequency=data2measure_peak_frequency.select(component="Z")
#     print(data2measure_peak_frequency[3:6])
#     data2measure_peak_frequency.taper(max_percentage=0.01,max_length=20)
#     # evt_data.trim(starttime=otime-20,endtime=otime+Td+20) 
#     #         # make plot of spectra
#     char_freq, sharp_weight= [],[]
#     fig1,ax1 = plt.subplots(1,1,figsize=(11,7), dpi = 200)
#     for i in range(len(data2measure_peak_frequency)):
#         data = data2measure_peak_frequency[i].data #*100
#         f,psd=scipy.signal.welch(data,fs=fs,nperseg=81,noverlap=4)
#         #just get the frequencies within the filter band
#         above_low_cut = [f>low_cut]
#         below_high_cut = [f<high_cut]
#         in_band = np.logical_and(above_low_cut,below_high_cut)[0]
#         f = f[in_band]
#         psd = psd[in_band]

#         # calculate characteristic frequency and report
#         char_freq_max = f[np.argmax(psd)]
#         char_freq_mean= np.sum(psd*f)/np.sum(psd)
#         psd_cumsum = np.cumsum(psd)
#         psd_sum = np.sum(psd)
#         char_freq_median = f[np.argmin(np.abs(psd_cumsum-psd_sum/2))]
#         char_freq.append(char_freq_mean)

# #             plt.rcParams.update({'font.size': 20})
#         ax1.plot(f,psd,label=stas[i],linewidth=2)
#         ax1.set_xscale('log')
#         ax1.set_yscale('log')
#         ax1.grid('True')
#         ax1.set_xlabel('Frequency [Hz]')
#         ax1.set_ylabel('PSD [$(mm/s)^2$/Hz]')
#         ax1.vlines(char_freq_mean,ymin=np.min(psd)/10,ymax=np.max(psd)*10,linestyle="--",colors=colors[i])

# #             # weighting the data by the spikiness of the PSD vs frequency graphs
#         ratio = (np.mean(psd)/np.max(psd))
#         sharp_weight.append(int(1/(ratio**2)*20))

# #         ax.legend() 
# #         plt.savefig('psd'+event_ID+associated_volcano+'.png')

#         # lats, lons, elevs, r, theta = ([] for i in range(5)) 
#         # ref = str(nets[0]+'.'+stas[0])
#         # try:
#         #     ref_env = data_env_dict[reference]
#         # except:
#         #     ref_env = data_env_dict[ref]


#     ################### PHASE PICKING using retrained eqT ####################
#     for i in range(len(sta_available)):
#         stream = evt_data.select(station=sta_available[i])
#         if len(stream)==3:
#             annotations = eqt.annotate(stream)
#             print(annotations)
#         elif len(stream)==1:
#             # copy stream to 2 components, zero the missing data.
#             tr3 = stream[0].copy()
#             tr2 = stream[0].copy();tr2.stats.channel=stream[0].stats.channel[0:2]+"N"
#             tr1 = stream[0].copy();tr1.stats.channel=stream[0].stats.channel[0:2]+"E"
#             tr1.data=np.zeros(len(stream[0].data))
#             tr2.data=np.zeros(len(stream[0].data))
#             crap=obspy.Stream(traces=[tr1,tr2,tr3])
#             print(crap)

#         pcount = {"P":0, "S":0}
#         for ii in eqt.classify(stream)[0]:
#             pcount[ii.phase] += 1
#         print('--------------'*3)
#         print(f"{pcount['P']} P picks\n{pcount['S']} S picks")

#         # print(__annotations__)
#         print(annotations[0])
#         annotations[0].stats.sampling_rate=40
#         t2=annotations[0].times()
#         ax.plot(t2,annotations[1].data+i*1.5, color = 'r')
#         ax.plot(t2,annotations[2].data+i*1.5, color = 'b')



#     plt.show()


In [13]:
print(evt_data)
tr3 = evt_data[-1].copy()
tr2 = evt_data[-1].copy();tr2.stats.channel=evt_data[-1].stats.channel[0:2]+"N"
tr1 = evt_data[-1].copy();tr1.stats.channel=evt_data[-1].stats.channel[0:2]+"E"
tr1.data=np.zeros(len(evt_data[-1].data))
tr2.data=np.zeros(len(evt_data[-1].data))
crap=obspy.Stream(traces=[tr1,tr2,tr3])
# crap[0].stats.channel=crap[0].stats.channel[0:2]+"I"
# crap[1].stats.channel=crap[0].stats.channel[0:2]+"H"
# crap[2].stats.channel=crap[0].stats.channel[0:2]+"Z"

# crap[0].data=np.zeros(len(crap[1].data))


print(crap[0].data)

# print(crap[2].data)

20 Trace(s) in Stream:
CC.ARAT..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:05:27.325000Z | 40.0 Hz, 1654 samples
CC.CARB..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.CRBN..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.OBSR..BHZ | 2023-11-05T07:04:46.005000Z - 2023-11-05T07:07:16.005000Z | 40.0 Hz, 6001 samples
CC.PANH..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.PARA..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.PR04..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.PR05..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.RUSH..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.SIFT..BHZ | 2023-11-05T07:04:46.000000Z - 2023-11-05T07:07:16.000000Z | 40.0 Hz, 6001 samples
CC.STYX

In [14]:
for n in range(6290,6291):    
    event_ID = str(evt_id[n])
    time = UTCDateTime(start_time[n])
    if net != 'CN' and evt_id[n]!=evt_id[n-1]:
        reference = str(net[n]+'.'+sta[n])
        try:
            associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
        except: 
            pass
        print(associated_volcano)
        # if associated_volcano == 'Mt_Rainier':
            
        #get info for stations within 50km of volcano that event ocurred at
        stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
        networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
        latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
        longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
        elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()

        if stations.count("LON")>0 and stations.count("LO2")>0:
            index = stations.index("LO2")
            del stations[index]
            del networks[index]
            del latitudes[index]
            del longitudes[index]
            del elevations[index]

            #Download all waveforms for that event based on stations and times
            bulk = [] 
            for m in range(0, len(networks)):
                bulk.append([networks[m], stations[m], '*', '*', time-t_before, time+t_after])
            st = client2.get_waveforms_bulk(bulk)

            #remove unwanted data
            for tr in st:
                cha = tr.stats.channel
                if cha[0:2] != 'BH' and cha[0:2] != 'EH' and cha[0:2] != 'HH':
                    st.remove(tr)
                try:
                    if len(tr.data)/tr.stats.sampling_rate < 239.9:
                        st.remove(tr)
                except:
                    pass

            #resampling the data to 40Hz for each trace
            st = resample(st,fs) 

            #Plotting all traces for one event with channel z, SNR>10, and bandpasses between 2-12Hz
            SNR,SNR_weight, no_weight,stas,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
            fig = plt.figure(figsize = (11,8), dpi=200)
            fig.suptitle('evtID:UW'+ event_ID+associated_volcano)
            plt.rcParams.update({'font.size': 20})
            ax1 = plt.subplot(1,1,1)
            iplot = 0
            for i,ii in enumerate(st):
                network = ii.stats.network
                station = ii.stats.station
                ii.detrend(type = 'demean')
                ii.filter('bandpass',freqmin=2.0,freqmax=12.0,corners=2,zerophase=True)
                cha = ii.stats.channel
                starttime = ii.stats.starttime
                max_amp_time = np.argmax(ii.data)/fs
                signal_window = ii.copy()
                noise_window = ii.copy()
                signal_window.trim(starttime + t_before - 20, starttime + t_before - 20 + window)
                noise_window.trim(starttime + t_before - window -10, starttime + t_before - 10)
                snr = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
                               / np.percentile(np.abs(noise_window.data),pr))/np.log(10))

                if cha[-1] == 'Z' and snr>thr and 100<max_amp_time<200:
                    t = ii.times()
                    t_diff[network+'.'+station] = starttime-time 
                    # enveloping the data 
                    data_envelope = obspy.signal.filter.envelope(ii.data[115*fs:150*fs])
                    data_envelope /= np.max(data_envelope)
                    data_envelope += iplot*1.5
                    # finding the time of max amplitude of each event
                    max_amp_times.append(max_amp_time)
                    max_amp = np.max(ii.data)      
                    # creating envelope data dictionary to calculate picktimes
                    data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
                    data_env_dict[network+'.'+station]= data_envelope
                    b,e = 115,150
                    ax1.plot(t[b*fs:e*fs],ii.data[b*fs:e*fs]/np.max(np.abs(ii.data))+iplot*1.5)
                    ax1.plot(t[115*fs:150*fs], data_envelope, color = 'k')
                    ax1.set_xlabel('time (seconds)')
                    ax1.set_xlim([b,e])
                    ax1.set_yticks([])
                    plt.text(t[e*fs], iplot*1.5, 'SNR:'+str(int(snr)))
                    plt.text(t[b*fs], iplot*1.5, station)
                    iplot = iplot+1
                    stas.append(ii.stats.station)
                    nets.append(ii.stats.network)
                    SNR.append(snr)
                    SNR_weight.append(int(snr))
                    no_weight.append(1)
                else:
                    st.remove(ii)

            if len(st)<4:  
                continue

            # get peak frequency of each event
            # read and preprocess data
            st.taper(max_percentage=0.01,max_length=20)
            st.trim(starttime=time-20,endtime=time+30) 

            # make plot of spectra
            char_freq, sharp_weight= [],[]
    #         fig,ax = plt.subplots(1,1,figsize=(11,7), dpi = 200)
            for i in range(len(stas)):
                data = st.select(station=stas[i],component="Z")[0].data*100
                f,psd=scipy.signal.welch(data,fs=st[0].stats.sampling_rate,nperseg=81,noverlap=1)
                #just get the frequencies within the filter band
                above_low_cut = [f>low_cut]
                below_high_cut = [f<high_cut]
                in_band = np.logical_and(above_low_cut,below_high_cut)[0]
                f = f[in_band]
                psd = psd[in_band]

                # calculate characteristic frequency and report
                char_freq_max = f[np.argmax(psd)]
                char_freq_mean= np.sum(psd*f)/np.sum(psd)
                psd_cumsum = np.cumsum(psd)
                psd_sum = np.sum(psd)
                char_freq_median = f[np.argmin(np.abs(psd_cumsum-psd_sum/2))]
                char_freq.append(char_freq_mean)

                plt.rcParams.update({'font.size': 20})
    #             ax.plot(f,psd,label=stas[i],linewidth=2)
    #             ax.set_xscale('log')
    #             ax.set_yscale('log')
    #             ax.grid('True')
    #             ax.set_xlabel('Frequency [Hz]')
    #             ax.set_ylabel('PSD [$(mm/s)^2$/Hz]')
    #             ax.vlines(char_freq_mean,ymin=np.min(psd)/10,ymax=np.max(psd)*10,linestyle="--",colors=colors[i])

                # weighting the data by the spikiness of the PSD vs frequency graphs
                ratio = (np.mean(psd)/np.max(psd))
                sharp_weight.append(int(1/(ratio**2)*20))

    #         ax.legend() 
    #         plt.savefig('psd'+event_ID+associated_volcano+'.png')

            lats, lons, elevs, r, theta = ([] for i in range(5)) 
            ref = str(nets[0]+'.'+stas[0])
            try:
                ref_env = data_env_dict[reference]
            except:
                ref_env = data_env_dict[ref]

            ############ PHASE PICKING ############################
            # calculating the picktimes and shift in arrival times using envelope cross_correlation
            pick_times, offsets, starttimes = pick_time(time, ref_env, data_env_dict,st,t_diff, t_before, fs) #calculate picktimes
            shifts, vals = shift(pick_times, offsets, starttimes, t_diff)

            iplot = 0 
            durations = []
            for i in range(len(stas)):
                max_amp_time = max_amp_times[i]
                duration = (max_amp_time-vals[i])*2
                durations.append(duration)
                ax1.vlines(vals[i], ymin = iplot*1.5-.5, ymax = iplot*1.5+.5, color = colors[i])
                #plt.text(t[110*fs], iplot*1.5, 'duration:'+str(int(duration))+'s')
                a = stations.index(stas[i])
                lats.append(latitudes[a])
                lons.append(longitudes[a])
                elevs.append(elevations[a])
                iplot = iplot+1
            avg_duration = np.mean(durations)
            plt.savefig('wiggles'+event_ID+associated_volcano+'.png')



            ############ LOCATION ############################
            # input necessary data for grid search
            arrivals = shifts
            sta_lats = lats
            sta_lons= lons

            # define grid origin in lat,lon and grid dimensions in m
            lat_start = volc_grid[associated_volcano][0]
            lon_start = volc_grid[associated_volcano][1]
            side_length = volc_grid[associated_volcano][2]

            # create the grid of locations
            sta_x = []
            sta_y = []
            for i in range(len(sta_lats)):
                x_dist = distance.distance([lat_start,lon_start],[lat_start,sta_lons[i]]).m
                y_dist = distance.distance([lat_start,lon_start],[sta_lats[i],lon_start]).m
                sta_x.append(x_dist)
                sta_y.append(y_dist)
            x_vect = np.arange(0, side_length, step)
            y_vect = np.arange(0, side_length, step)
            t0 = np.arange(0,np.max(arrivals),t_step)

            # carry out the gridsearch weighted by SNR
            weight = np.array(SNR_weight)/np.max(SNR_weight)
            rss_mat = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,weight)
            loc_idx_snr = np.unravel_index([np.argmin(rss_mat)], rss_mat.shape)

            # gridsearch with no weight
            weight = [1 for i in range(len(SNR_weight))]
            rss_mat = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,weight)
            loc_idx = np.unravel_index([np.argmin(rss_mat)], rss_mat.shape)

            # gridsearch weighted with SNR and Slope
            # plot DEM
            # gives the lower left grid point in the grid search
            # gives the left right, bottom, top of the grid
            left, right = r_dem_data_dict[associated_volcano]['left'],r_dem_data_dict[associated_volcano]['right']
            bottom, top = r_dem_data_dict[associated_volcano]['bottom'],r_dem_data_dict[associated_volcano]['top']

            crs = dem_data_dict[associated_volcano]['crs']
            data = dem_data_dict[associated_volcano]['data']
            volc = rd.rdarray(data, no_data=-9999)
            slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
            info = volc_lat_lon[associated_volcano]
            p2 = Proj(crs,preserve_units=False)
            p1 = Proj(proj='latlong',preserve_units=False)
            # gives the lower left grid point in the grid search
            left_x,bottom_y = transform(p1,p2,volc_grid[associated_volcano][1],volc_grid[associated_volcano][0]) # p1,p2,lon,lat
            # gives the left right, bottom, top of the grid
            grid_bounds = [left_x, left_x+volc_grid[associated_volcano][2], bottom_y, bottom_y+volc_grid[associated_volcano][2]]
            left, right = r_dem_data_dict[associated_volcano]['left'],dem_data_dict[associated_volcano]['right']
            bottom, top = r_dem_data_dict[associated_volcano]['bottom'],dem_data_dict[associated_volcano]['top']

            a = int((left_x-left)/10)
            b = a+2500
            c = (slope.shape[0] - int((bottom_y-bottom)/10))-2500
            d = slope.shape[0] - int((bottom_y-bottom)/10)

            x = np.arange(a,b,1)
            y = np.arange(c,d,1)

            x2 = np.arange(a,b,10) # every 100m
            y2 = np.arange(c,d,10) # every 100m

            slope_data = np.array(slope[c:d,a:b])


            slope_norm1 = slope_data/np.max(slope_data)

            slope_interp_mat = RectBivariateSpline(y,x,slope_norm1, s = 0)
            interp = slope_interp_mat(x2,y2)/np.max(slope_interp_mat(x2,y2))*0.9+.1

            rss_mat_slope = np.multiply(rss_mat[loc_idx[0],:,:],interp)
            loc_idx_slope = np.unravel_index([np.argmin(rss_mat_slope)], rss_mat_slope.shape)
            loc_lat_slope, loc_lon_slope, test_d = location(x_vect[loc_idx_slope[1]], y_vect[loc_idx_slope[2]], lat_start, lon_start)

            # plot heatmap
    #         fig,ax = plt.subplots(1,1,figsize=(8,8), dpi = 200)
    #         ax.scatter(x_vect[loc_idx[1]],y_vect[loc_idx[2]],s=100,marker='*',c='r')
    #         im = ax.imshow(np.log10(rss_mat[loc_idx[0],:,:].T),origin="lower",extent=[0,side_length,0,side_length])
    #         ax.set_ylabel('(m)')
    #         ax.set_ylabel('(m)')
    #         cbar = plt.colorbar(im)
    #         cbar.ax.tick_params()
    #         cbar.set_label('RMS error on location', rotation=270)
    #         plt.savefig('heatmap'+ event_ID+associated_volcano+'.png')


            # find the latitude and longitude of the location index
            loc_lat, loc_lon, d = location(x_vect[loc_idx[1]], y_vect[loc_idx[2]], lat_start, lon_start)
            err_thr = np.min(np.log10(rss_mat))+.05
            thr_array = np.argwhere(np.log10(rss_mat)<err_thr)
            diameter = error_diameter(thr_array)

            break
            # calculating azimuth for each station with respect to the middle of the volcano
            for i in range(len(stas)):
                u,b,c = (gps2dist_azimuth(loc_lat, loc_lon, lats[i], lons[i], a=6378137.0, f=0.0033528106647474805))
                r.append(u)
                theta.append(b)

            bin1,bin2,bin3 = [],[],[]
            for i in theta:
                if 0<=i<=120:
                    bin1.append(i)
                if 121<=i<=240:
                    bin2.append(i)
                if 241<=i<=360:
                    bin3.append(i)

            if bin1 == [] or bin2 == [] or bin3 == []:
                continue

            #manipulating the data
            data = {'azimuth_deg':theta, 'freq':char_freq, 'station':stas, 'distance_m':r, 
                    'weight':sharp_weight, 'SNR':SNR, 'colors':colors[0:len(stas)]}
            DF = pd.DataFrame(data, index = None)
            DF2 = DF.sort_values('azimuth_deg')

            #Taking out stations that are too close to the location when looking at azimuth 
            drops = []
            for i in range(len(DF2)):
                value = DF2.loc[i,'distance_m']
                if value < az_thr:
                    drops.append(i)
            DF3 = DF2.drop(drops)
            y_data =  DF3["freq"].values.tolist()
            Sta2 = DF3["station"].values.tolist()
            dist2 = DF3["distance_m"].values.tolist()
            spike_weight = DF3["weight"].values.tolist()
            SNR2 = DF3['SNR'].values.tolist()
            colors2 = DF3['colors'].values.tolist()
            x_data =  np.asarray(DF3["azimuth_deg"].values.tolist())
            x_points = np.linspace(0,360, 100)


            ################ DIRECTIVITY FIT ##################################
            #optimizing parameters to fit data to test_function
            params, params_covariance = optimize.curve_fit(test_func, np.deg2rad(x_data), y_data, p0=None)
            perr = np.sqrt(np.diag(params_covariance))
            std_deviation = str(round(perr[0],9))+','+str(round(perr[1],9))+','+str(round(perr[2],9))
            d = test_func(np.deg2rad(x_points), params[0], params[1], params[2])
            len_r = int(max(r))

            if params[0]<0:
                direction = params[1]+pi 
            else:
                direction = params[1]

            fmax = max(d)
            fmin = min(d)
            v = v_s*((fmax-fmin)/(fmax+fmin))



            # weight the data
            # title = 'Sharpness'
            v_sharp,direction_sharp,d_sharp = weight_data(x_data,y_data,sharp_weight,test_func,v_s,stas)

            # title = 'SNR'
            v_snr,direction_snr,d_snr = weight_data(x_data,y_data,SNR_weight,test_func,v_s,stas)  




            ############### PLOTS ###################
            #convert the direction from polar to cartesian coordinates
            dy = len_r*np.sin(direction)
            dx = len_r*np.cos(direction)     

            dy_sharp = len_r*np.sin(direction_sharp)
            dx_sharp = len_r*np.cos(direction_sharp)    


            dy_snr = len_r*np.sin(direction_snr)
            dx_snr = len_r*np.cos(direction_snr) 




            fig,ax = plt.subplots(1,1,figsize=(11,8), dpi = 200)
            fig.suptitle('Fitted Cosine Curves')       
            ax.set_ylabel('characteristic frequency(Hz)')
            ax.set_xlabel(('azimuth(degrees)'))
            for i in range (0,len(Sta2)):
                ax.scatter(x_data[i], y_data[i], s = (SNR_weight[i]**2),label=Sta2[i], color = colors2[i])
            ax.plot(x_data,y_data, '--', label='rawdata')
            ax.plot(x_points, d, label = 'original')
            ax.plot(x_points, d_sharp, label = 'sharpness')
            ax.plot(x_points, d_snr, label = 'snr')
            ax.legend(loc='upper right', fontsize = 10)
            plt.grid(True)
            plt.savefig('curves_freq_data'+ event_ID+associated_volcano+'.png')

            #making plots of directivity and location
            crs = dem_data_dict[associated_volcano]['crs']
            data = dem_data_dict[associated_volcano]['data']
            info = volc_lat_lon[associated_volcano]
            p2 = Proj(crs,preserve_units=False)
            p1 = Proj(proj='latlong',preserve_units=False)
            # gives the lower left grid point in the grid search
            left_x,bottom_y = transform(p1,p2,volc_grid[associated_volcano][1],volc_grid[associated_volcano][0]) # p1,p2,lon,lat
            # gives the left right, bottom, top of the grid
            grid_bounds = [left_x, left_x+volc_grid[associated_volcano][2], bottom_y, bottom_y+volc_grid[associated_volcano][2]]
            left, right = dem_data_dict[associated_volcano]['left'],dem_data_dict[associated_volcano]['right']
            bottom, top = dem_data_dict[associated_volcano]['bottom'],dem_data_dict[associated_volcano]['top']

            # convert loc data onto the DEM data
            contour_x,contour_y = np.meshgrid(left_x+x_vect,bottom_y+y_vect)
            center_x, center_y = transform(p1,p2,info[1],info[0])
            loc_x,loc_y=transform(p1,p2,loc_lon,loc_lat)
            duration=avg_duration
            length_factor = duration/100

            fig,ax = plt.subplots(1,1,figsize=(8,11), dpi = 200)
            a = ax.imshow(data,extent=[left, right, bottom, top],cmap='gist_earth')
            contours = ax.contour(contour_x,contour_y,np.log10(rss_mat[int(loc_idx[0]),:,:].T),cmap='plasma')
            ax.scatter(center_x, center_y, s=100,marker='*',c='r')
            plt.arrow(loc_x,loc_y,dy*length_factor,dx*length_factor, color='w', width=170, label='no weight')
            plt.arrow(loc_x,loc_y,dy_sharp*length_factor,dx_sharp*length_factor, color='k', width=170, label='sharpness')
            plt.arrow(loc_x,loc_y,dy_snr*length_factor,dx_snr*length_factor, color='m', width=170, label='snr')
            #plotting the stations on top of this as triangles
            for i, ii in enumerate(stas):
                sta_x,sta_y = transform(p1,p2,lons[i],lats[i])
                if left+info[3]<sta_x<right-info[4] and bottom+info[5]<sta_y<top-info[6]:
                    ax.plot(sta_x,sta_y, c='k', marker="^")
                    ax.text(sta_x,sta_y,ii, c='k', fontsize = 15)

            #getting lat and lon tick marks on the axis
            tick_lons = lat_lon_dict[associated_volcano]['tick_lons']
            tick_lats = lat_lon_dict[associated_volcano]['tick_lats']
            ticks_x = []
            ticks_y = []
            for i in range(len(tick_lons)):
                tick_x,tick_y=transform(p1,p2,tick_lons[i],tick_lats[i])
                ticks_x.append(tick_x)
                ticks_y.append(tick_y)
                tick_lons[i]=str(tick_lons[i])
                tick_lats[i]=str(tick_lats[i])
            divider = make_axes_locatable(ax)
            cax1 = divider.append_axes('right', size='4%', pad=0.1)
            cax2 = divider.append_axes('right', size='4%', pad=1.3)
            ax.set_title('Location and Directivity', fontsize = 20)
            ax.set_xlabel('longitudes(DD)', fontsize = 15)
            ax.set_ylabel('latitudes(DD)', fontsize = 15)
            ax.set_xticks(ticks_x)
            ax.set_xticklabels(tick_lons, fontsize = 15)
            ax.set_yticks(ticks_y)
            ax.set_yticklabels(tick_lats, fontsize = 15)
            ax.clabel(contours)
            cbar = plt.colorbar(a, cax=cax1)
            cbar.ax.tick_params(labelsize=10)
            cbar.set_label('elevation(m)\n', rotation=270, labelpad = 13, fontsize = 15)
            cbar2 = plt.colorbar(contours, cax=cax2)
            cbar2.ax.tick_params(labelsize=10)
            cbar2.set_label('RMS error on location\n', rotation=270, labelpad = 13,fontsize = 15)
            ax.set_xlim(left+info[3],right-info[4])
            ax.set_ylim(bottom+info[5],top-info[6])
            ax.legend(fontsize = 12)
            plt.tight_layout()
            plt.savefig('loc_direction'+ event_ID+associated_volcano+'.png',bbox_inches="tight")

            # make a dataframe of the data
            evt_data = evt_data.append({'event_ID':event_ID, 
                        'location_latitude': loc_lat,
                        'location_longitude': loc_lon,
                        'location_uncertainty(m)':diameter/10,
                        'origin_time': min(offsets)-int(loc_idx[0]),
                        'direction(degrees)':np.rad2deg(direction),
                        'direction_sharpness(degrees)':np.rad2deg(direction_sharp),
                        'direction_snr(degrees)':np.rad2deg(direction_snr),
                        'duration(sec)':avg_duration,
                        'params_std_deviation':std_deviation, 
                        'velocity(m/s)':v, 
                        'number_of_stations':len(stas)}, ignore_index = True)

            dict_temp = {}
            for i in range(len(stas)):
                dict_temp[stas[i]] = char_freq[i]
            print(dict_temp)    
            sta_freq = sta_freq.append(dict_temp,ignore_index = True)

            evt_data.to_csv('~/surface_events/Event_Data.csv', index=False)
            sta_freq.to_csv('~/surface_events/Station_frequency_data.csv', index=False)
    #     except:
    #         reject_evts = reject_evts.append({'event_ID':[event_ID]}, ignore_index = True)
    #         reject_evts.to_csv('~/surface_events/Rejects5.csv', index=False)
    #         continue


Mt_St_Helens


In [15]:
#DEM data 
r_dem_data_dict = {}
name = 'Mt_Rainier'
if volc_lat_lon[name][0]>46:
    dem = rio.open('Data/DEM_data/'+str(name)+'/'+str(name)+'1.tif') #washington volcanoes
    dem_array = dem.read(1).astype('float64')
    dem_array[dem_array == -32767] = np.nan #gets rid of edge effects
    crs = dem.crs

r_dem_data_dict[name]={'data':dem_array, 'crs':crs, 'left':dem.bounds[0], 'right':dem.bounds[2], 'bottom':dem.bounds[1], 'top':dem.bounds[3]}

NameError: name 'volc_lat_lon' is not defined

In [ ]:
# Determine the avg slope value within one step of the grid search model
associated_volcano = 'Mt_Rainier'
step_gs = 100 # grid search model step size is 100m

# plot DEM
crs = r_dem_data_dict[associated_volcano]['crs']
data = r_dem_data_dict[associated_volcano]['data']
volc = rd.rdarray(data, no_data=-9999)
slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
info = volc_lat_lon[associated_volcano]
p2 = Proj(crs,preserve_units=False)
p1 = Proj(proj='latlong',preserve_units=False)
# gives the lower left grid point in the grid search
left_x,bottom_y = transform(p1,p2,volc_grid[associated_volcano][1],volc_grid[associated_volcano][0]) # p1,p2,lon,lat
# gives the left right, bottom, top of the grid
grid_bounds = [left_x, left_x+volc_grid[associated_volcano][2], bottom_y, bottom_y+volc_grid[associated_volcano][2]]
left, right = r_dem_data_dict[associated_volcano]['left'],r_dem_data_dict[associated_volcano]['right']
bottom, top = r_dem_data_dict[associated_volcano]['bottom'],r_dem_data_dict[associated_volcano]['top']
contour_x,contour_y = np.meshgrid(left_x+x_vect,bottom_y+y_vect)
center_x, center_y = transform(p1,p2,info[1],info[0])
loc_x,loc_y=transform(p1,p2,loc_lon,loc_lat)

# looking at where the grid is:
x = np.linspace(left_x, left_x+25000, 10)
y = np.linspace(bottom_y, bottom_y+25000, 10)

In [ ]:
a = int((left_x-left)/10)
b = a+2500
c = (slope.shape[0] - int((bottom_y-bottom)/10))-2500
d = slope.shape[0] - int((bottom_y-bottom)/10)

x = np.arange(a,b,1)
y = np.arange(c,d,1)

x2 = np.arange(a,b,10) # every 100m
y2 = np.arange(c,d,10) # every 100m

slope_data = np.array(slope[c:d,a:b])

In [ ]:
slope_norm1 = slope_data/np.max(slope_data)

slope_interp_mat = RectBivariateSpline(y,x,slope_norm1, s = 0)
interp = slope_interp_mat(x2,y2)/np.max(slope_interp_mat(x2,y2))*0.9+.1

rss_mat_slope = np.multiply(rss_mat[loc_idx[0],:,:],interp)
loc_idx_slope = np.unravel_index([np.argmin(rss_mat_slope)], rss_mat_slope.shape)
loc_lat_slope, loc_lon_slope, j = location(x_vect[loc_idx_slope[1]], y_vect[loc_idx_slope[2]], lat_start, lon_start)


In [ ]:
plt.rcParams.update({'font.size': 5})

# weighted by SNR
weight = np.array(SNR_weight)/np.max(SNR_weight)
rss_mat = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,weight)
loc_idx = np.unravel_index([np.argmin(rss_mat)], rss_mat.shape)
loc_lat, loc_lon, j = location(x_vect[loc_idx[1]], y_vect[loc_idx[2]], lat_start, lon_start)

x2 = np.linspace(a,b,250)
y2 = np.linspace(c,d,250)

# weighted by SNR and Slope
rss_mat_slope = np.multiply(rss_mat[loc_idx[0],:,:],interp)
loc_idx_slope = np.unravel_index([np.argmin(rss_mat_slope)], rss_mat_slope.shape)
loc_lat_slope, loc_lon_slope, j = location(x_vect[loc_idx_slope[1]], y_vect[loc_idx_slope[2]], lat_start, lon_start)

# unweighted
no_weight = [1,1,1,1,1,1,1,1]
rss_mat_nw = gridsearch(t0,x_vect,y_vect,sta_x,sta_y,1000,arrivals,no_weight)
loc_idx_nw = np.unravel_index([np.argmin(rss_mat_nw)], rss_mat_nw.shape)
loc_lat_nw, loc_lon_nw, j = location(x_vect[loc_idx_nw[1]], y_vect[loc_idx_nw[2]], lat_start, lon_start)


# plotting the results
fig,ax = plt.subplots(3,1,figsize=(6,9), dpi = 180)

ax[0].set_title('weighted by SNR: '+str(round(loc_lat,3))+','+str(round(loc_lon,3)))
ax[0].scatter(x_vect[loc_idx[1]],y_vect[loc_idx[2]],s=100,marker='*',c='r')
im0 = ax[0].imshow(np.log10(rss_mat[loc_idx[0],:,:].T),origin="lower",extent=[0,side_length,0,side_length])
#im0 = ax[0].imshow(rss_mat[loc_idx[0],:,:].T,origin="lower",extent=[0,side_length,0,side_length])
ax[0].set_ylabel('(m)')
ax[0].set_ylabel('(m)')
cbar0 = plt.colorbar(im0, ax = ax[0])
cbar0.ax.tick_params()
cbar0.set_label('RMS error on location', rotation=270)

ax[1].set_title('weighted by SNR and Slope: '+str(round(loc_lat_slope,3))+','+str(round(loc_lon_slope,3)))
ax[1].scatter(x_vect[loc_idx_slope[1]],y_vect[loc_idx_slope[2]],s=100,marker='*',c='r')
im1 = ax[1].imshow(np.log10(rss_mat_slope[0,:,:].T),origin="lower",extent=[0,side_length,0,side_length])
#im1 = ax[1].imshow(rss_mat_slope[0,:,:].T,origin="lower",extent=[0,side_length,0,side_length])
ax[1].set_ylabel('(m)')
ax[1].set_ylabel('(m)')
cbar1 = plt.colorbar(im1, ax = ax[1])
cbar1.ax.tick_params()
cbar1.set_label('RMS error on location', rotation=270)

ax[2].set_title('unweighted: '+str(round(loc_lat_nw,3))+','+ str(round(loc_lon_nw,3)))
ax[2].scatter(x_vect[loc_idx_nw[1]],y_vect[loc_idx_nw[2]],s=100,marker='*',c='r')
im2 = ax[2].imshow(np.log10(rss_mat_nw[0,:,:].T),origin="lower",extent=[0,side_length,0,side_length])
#im2 = ax[2].imshow(rss_mat_nw[0,:,:].T,origin="lower",extent=[0,side_length,0,side_length])
ax[2].set_ylabel('(m)')
ax[2].set_ylabel('(m)')
cbar2 = plt.colorbar(im2, ax = ax[2])
cbar2.ax.tick_params()
cbar2.set_label('RMS error on location', rotation=270)